In [101]:
from nltk import ngrams
import cPickle
from scipy.sparse import hstack
import re


def string_feature_gen(x):
    temp=x
    temp="".join(re.findall(r"\w+",x.lower().replace("http",".").replace("www",".")))
    temp=[" ".join(["".join(gram) for gram in ngrams(temp, 3)])]
    with open('feature_genrator_special_charac.pkl', 'rb') as fid:
        vec_special=cPickle.load(fid)
    with open('feature_genrator.pkl', 'rb') as fid:
        vec_aplha=cPickle.load(fid)
    special_characters=['#','/', '%', '~', '=','?','&',"-","(",")","+","@","[","]","'","*"]
    x_alpha_features=vec_aplha.transform(temp)
    temp=" ".join(re.findall("[^\:\w\.]",x.replace("http://|https://","")))
    for n,spcl in enumerate(special_characters):
        temp=temp.replace(spcl,str(n+10))
    x_special_features=vec_special.transform([temp])
    return hstack([x_alpha_features,x_special_features,len(x)])


def predict(url):
    with open('svm_with_higher_recall.pkl', 'rb') as fid:
        clf_loaded = cPickle.load(fid)
    y_pred=clf_loaded.predict(string_feature_gen(url))
    """if y_pred==1:
        print("it is a profile")
    else:
        print("sorry, not profile as far as i have learnt")
    """
    return bool(y_pred)
